In [ ]:
! pip install tensorflow numpy


  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached markdown-3.8

In [ ]:
data = [
    ('hello', 'bonjour'),
    ('how are you', 'comment ça va'),
    ('i am fine', 'je vais bien'),
    ('thank you', 'merci'),
    ('goodbye', 'au revoir')
]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Split into input and target language
eng_sentences, fr_sentences = zip(*data)
fr_sentences = ['<start> ' + s + ' <end>' for s in fr_sentences]

# Tokenize
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(eng_sentences)
eng_seq = eng_tokenizer.texts_to_sequences(eng_sentences)
eng_pad = pad_sequences(eng_seq, padding='post')

fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(fr_sentences)
fr_seq = fr_tokenizer.texts_to_sequences(fr_sentences)
fr_pad = pad_sequences(fr_seq, padding='post')

# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, 64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(64, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(fr_vocab_size, 64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
output = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


In [ ]:
fr_input = fr_pad[:, :-1]
fr_target = fr_pad[:, 1:]
fr_target = np.expand_dims(fr_target, -1)


In [ ]:
model.fit([eng_pad, fr_input], fr_target, epochs=500, verbose=0)
print("Training complete.")


In [ ]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_emb2 = Embedding(fr_vocab_size, 64)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [ ]:
def translate(input_text):
    seq = eng_tokenizer.texts_to_sequences([input_text])
    pad = pad_sequences(seq, maxlen=eng_pad.shape[1], padding='post')
    states_value = encoder_model.predict(pad)

    target_seq = np.array([[fr_tokenizer.word_index['<start>']]])
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence.split()) > 10:
            break

        decoded_sentence += ' ' + sampled_word
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return decoded_sentence.strip()


In [ ]:
print(translate("hello"))         # → bonjour
print(translate("how are you"))   # → comment ça va
